# FVCA

In [ ]:
from trustutils import run

run.introduction("Yannick Gorsse")
run.description("")
run.TRUST_parameters()

In [ ]:
from trustutils import run
from string import Template
import os, sys
run.useMEDCoupling()
import medcoupling as mc

nb_meshes_max = 2 # mettre a 10 si on veut faire tous les maillages

run.reset()
m2d = {
    # 2D meshes
    "Cartesian":      (["PolyMAC", "PolyMAC_P0"], 7), 
    "Triangles":      (["PolyMAC", "PolyMAC_P0"], 6),
    "Locally_Refined":(["PolyMAC", "PolyMAC_P0"], 7),
    "Quadrangles":    (["PolyMAC", "PolyMAC_P0"], 7),
    "Kershaw2D":      (["PolyMAC", "PolyMAC_P0"], 6),
    "Polygons":       (["PolyMAC", "PolyMAC_P0"], 6),
}
m3d = {
    # 3D meshes
    "Hexa":           (["PolyMAC", "PolyMAC_P0"], 5), # TODO : ajouter VDF, mais veut pas unpolyze...
    "Tetra":          (["PolyMAC", "PolyMAC_P0"], 7),
    "Prism":          (["PolyMAC", "PolyMAC_P0"], 4),
    "Kershaw":        (["PolyMAC", "PolyMAC_P0"], 4),
    "CheckerBoard":   (["PolyMAC", "PolyMAC_P0"], 5),
    "PrismHexa":      (["PolyMAC", "PolyMAC_P0"], 4),
    "Random":         (["PolyMAC", "PolyMAC_P0"], 4),
    "LocRaf":         (["PolyMAC", "PolyMAC_P0"], 5),
    "Voronoi":        (["PolyMAC", "PolyMAC_P0"], 5),
}

meshes = {**m2d, **m3d}

meshes_dir = os.environ["TRUST_ROOT"] + "/Validation/Rapports_automatiques/Verification/PolyMAC/share/meshes"

# Trigger the copy of the src directory into the build directory:
run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)
origin, build_dir = os.getcwd(), run.defaultSuite_.getBuildDirectory()
os.chdir(build_dir)
list_dis = []
for m, (dis, ns) in meshes.items():
    for d in dis:
        for n in range(1, min(ns, nb_meshes_max) + 1):
            os.system(f"mkdir -p {m}/{d}/jdd_{n}")
            jdd = "jdd_2d.data" if m in m2d else "jdd_3d.data"
            with open(jdd, "r") as file: filedata = Template(file.read())
            result = filedata.substitute({"dis" : d})
            with open(f"{m}/{d}/jdd_{n}/jdd.data", "w") as file: file.write(result)  
            if not d.startswith("Poly"):
                os.system(f"gunzip -c {meshes_dir}/{m}/jdd_{n}/mesh.med.gz >{m}/{d}/jdd_{n}/mesh.med")
                meshname = f"{m}/{d}/jdd_{n}/mesh.med"
                mfum = mc.MEDFileUMesh(meshname)
                mfum.getMeshAtLevel(0).unPolyze()
                mfum.write(f"{m}/{d}/jdd_{n}/mesh.med", 2)
            else: os.system(f"gunzip -c {meshes_dir}/{m}/jdd_{n}/mesh.med.gz >{m}/{d}/jdd_{n}/mesh.med")
            os.system(f"cp post_run {m}/{d}/jdd_{n}/")
            #echo $mesh/jdd_${n}/cas.data >> ll
            if not m.startswith("Prism") or n!= 2 :
                run.addCase(f"{m}/{d}/jdd_{n}", "jdd.data")
                list_dis.append(f"{m}/{d}")

with open("extract_convergence", "r") as file: fileconv = Template(file.read())
result = fileconv.safe_substitute(list_dis=" ".join(sorted(list(set(list(list_dis))))))
with open("extract_convergence", "w") as file: file.write(result)
os.chdir(origin)

run.printCases()
run.runCases()
run.executeScript("extract_convergence")

In [ ]:
from trustutils import plot
import matplotlib.colors as mcolors
import pandas as pd
c = list(mcolors.TABLEAU_COLORS.keys())
linestyle = {"PolyMAC": ("solid", "o"), "VDF" : ("dotted", "*"), "PolyMAC_P0" : ("dashed", "v"), "VEFPreP1B" : ("dashdot", "s")}

df = run.tablePerf()

for dim, dic in [(2, m2d), (3, m3d)]:
    a = plot.Graph(f"Temps de calcul {dim}D", nY=2)
    for k, (m, (dis, _)) in enumerate(dic.items()):
        df_m = df[df.index.map(lambda s: m in s)]
        for d in dis:
            df_md = df_m[df_m.index.map(lambda s: d in s)]
            for i, metrique in enumerate(["CPU time/step", "Total CPU Time"]):
                a.addPlot(i)
                nbc = [float(n) for n in df_md["number of cells"].tolist()]
                tcpu = [float(t) for t in df_md[metrique].tolist()]
                a.add(nbc, tcpu, title=f"Maillages {dim}D", label=f"{m}-{d}", color=c[k], linestyle=None, marker=linestyle[d][1], markersize=12)
                a.label(f"Number of cells", metrique)


## Ordres de convergence

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def func(x, a, b): return a * x ** b

for dim, dic in [(2, m2d), (3, m3d)]:
    a = plot.Graph(f"Maillages {dim}D")
    for k, (m, (dis, ns)) in enumerate(dic.items()):
        for d in dis:
            x_ = plot.loadText(f"{m}/{d}/conv_glob")
            x, y = (np.power(x_[0], -1.0 / dim)).tolist(), x_[1].tolist()
            (_, b), _ = curve_fit(func, x, y)
            a.add(x, y, label=f"{m}-{d}, order={b:.2f}", color=c[k], linestyle=linestyle[d][0], marker=linestyle[d][1], markersize=12)

    a.label(f"$Ne^{{-1/{dim}}}$","erreur L2")
    a.scale(xscale='log',yscale='log')


In [ ]:
for dim, dic in [(2, m2d), (3, m3d)]:
    a = plot.Graph(f"Maillages {dim}D")
    for k, (m, (dis, _)) in enumerate(dic.items()):
        for d in dis:
            if not m.startswith("Prism") or n!= 2 :
                x_ = plot.loadText(f"{m}/{d}/conv_glob")
                x, y = x_[2].tolist(), x_[1].tolist()
                a.add(x, y, label=f"{m}-{d}", color=c[k], linestyle=linestyle[d][0], marker=linestyle[d][1], markersize=12)

    a.label(f"Temps de calcul","erreur L2")
    a.scale(xscale='log',yscale='log')
